In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

In [22]:
df_raw = pd.read_csv("../data/rbi_money_circulation.csv")

In [23]:
df_raw.head()

,year,Coins - 0000.5,Coins - 0001,Coins - 0002,Coins - 0005,Coins - 0010,Notes - 0001,Notes - 0002,Notes - 0005,Notes - 0010,Notes - 0020,Notes - 0050,Notes - 0100,Notes - 0500,Notes - 1000
0,1977,1.97,0.96,NaN,NaN,NaN,2.72,1.48,5.72,18.25,6.90,9.95,43.02,NaN,0.55
1,1978,2.09,1.10,NaN,NaN,NaN,2.56,1.74,5.76,19.57,7.79,14.40,53.69,NaN,NaN
2,1979,2.25,1.26,NaN,NaN,NaN,2.14,2.15,5.88,21.23,9.61,19.83,59.51,NaN,NaN
3,1980,2.43,1.41,NaN,NaN,NaN,2.07,2.99,6.34,20.97,10.90,22.06,74.06,NaN,NaN
4,1981,2.63,1.54,NaN,NaN,NaN,2.12,3.92,6.92,19.91,12.14,27.24,78.29,NaN,NaN


In [24]:
df_raw.dtypes

year                int64
Coins - 0000.5    float64
Coins - 0001      float64
Coins - 0002      float64
Coins - 0005      float64
Coins - 0010      float64
Notes - 0001      float64
Notes - 0002      float64
Notes - 0005      float64
Notes - 0010      float64
Notes - 0020      float64
Notes - 0050      float64
Notes - 0100      float64
Notes - 0500      float64
Notes - 1000      float64
dtype: object

## Fill Missing Values for 1 Rupee Notes and Coins

In [25]:
df_raw['Notes - 0001'] = df_raw['Notes - 0001'].ffill()

In [26]:
df_raw['Coins - 0001'] = df_raw['Coins - 0001'].ffill()

## To Wide to Tall Conversion

In [27]:
df_raw_1 = pd.melt(df_raw, id_vars=['year'], var_name='currency', value_name='money')

In [28]:
df_raw_1.head()

,year,currency,money
0,1977,Coins - 0000.5,1.97
1,1978,Coins - 0000.5,2.09
2,1979,Coins - 0000.5,2.25
3,1980,Coins - 0000.5,2.43
4,1981,Coins - 0000.5,2.63


In [29]:
df_raw_1['type'] = df_raw_1.currency.str.split(' - ').str[0]
df_raw_1['denom'] = df_raw_1.currency.str.split(' - ').str[1]

In [30]:
df_raw_1.tail()

,year,currency,money,type,denom
541,2011,Notes - 1000,3468.81,Notes,1000
542,2012,Notes - 1000,4299.00,Notes,1000
543,2013,Notes - 1000,5081.37,Notes,1000
544,2014,Notes - 1000,5612.45,Notes,1000
545,2015,Notes - 1000,6325.68,Notes,1000


In [31]:
df = df_raw_1[df_raw_1['type'] == "Notes"].copy()

In [32]:
df.denom.unique()

array(['0001', '0002', '0005', '0010', '0020', '0050', '0100', '0500',
       '1000'], dtype=object)

In [33]:
df['value'] = pd.to_numeric(df['denom'])

In [34]:
df['year'] = pd.to_numeric(df['year'])

In [35]:
df.dtypes

year          int64
currency     object
money       float64
type         object
denom        object
value         int64
dtype: object

In [36]:
df['number'] = (df['money']/df['value']).round(3)

In [37]:
df.head()

,year,currency,money,type,denom,value,number
195,1977,Notes - 0001,2.72,Notes,0001,1,2.72
196,1978,Notes - 0001,2.56,Notes,0001,1,2.56
197,1979,Notes - 0001,2.14,Notes,0001,1,2.14
198,1980,Notes - 0001,2.07,Notes,0001,1,2.07
199,1981,Notes - 0001,2.12,Notes,0001,1,2.12


In [38]:
df = df.sort_values(by="year")

In [39]:
df.drop(['currency'], axis = 1, inplace=True)

In [40]:
df.to_csv("../data/notes.csv", index=False)

## Only 2015 Data

In [41]:
df.head()

,year,money,type,denom,value,number
195,1977,2.72,Notes,0001,1,2.720
507,1977,0.55,Notes,1000,1000,0.001
234,1977,1.48,Notes,0002,2,0.740
390,1977,9.95,Notes,0050,50,0.199
351,1977,6.90,Notes,0020,20,0.345


In [42]:
df2015 = df[df.year == 2015]

In [45]:
df2015_small = df2015[['year', 'denom', 'money', "number"]]

In [47]:
df2015_small.to_csv("../data/notes2015.csv", index=False)

## Pivot to get a row wise output

In [57]:
df['year'] = "Y" + df['year'].astype(str)

In [58]:
dfWide = pd.pivot_table(df, values = "money", columns = "year", index = "denom")

In [59]:
dfWide.head()

year,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,...,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015
denom,,,,,,,,,,,,,,,,,,,,,
0001,2.72,2.56,2.14,2.07,2.12,2.05,1.98,2.04,2.38,3.01,...,3.01,3.00,3.00,2.99,2.99,2.99,2.99,2.99,2.99,3.09
0002,1.48,1.74,2.15,2.99,3.92,4.31,4.50,5.80,6.50,6.83,...,4.47,6.36,6.65,6.98,8.51,8.51,8.51,8.51,8.54,8.53
0005,5.72,5.76,5.88,6.34,6.92,8.07,5.60,12.34,13.03,12.87,...,18.87,21.11,22.71,22.33,34.30,36.43,36.87,37.14,37.02,36.80
0010,18.25,19.57,21.23,20.97,19.91,17.98,17.70,22.17,25.36,25.10,...,71.55,93.33,122.22,185.36,212.88,230.02,251.68,266.48,303.04,320.15
0020,6.90,7.79,9.61,10.90,12.14,14.33,14.63,16.36,17.84,18.99,...,41.78,41.08,43.99,46.81,60.40,70.20,76.50,85.69,86.99,98.47


In [60]:
dfWide.reset_index()

year,denom,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,...,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015
0,0001,2.72,2.56,2.14,2.07,2.12,2.05,1.98,2.04,2.38,...,3.01,3.00,3.00,2.99,2.99,2.99,2.99,2.99,2.99,3.09
1,0002,1.48,1.74,2.15,2.99,3.92,4.31,4.50,5.80,6.50,...,4.47,6.36,6.65,6.98,8.51,8.51,8.51,8.51,8.54,8.53
2,0005,5.72,5.76,5.88,6.34,6.92,8.07,5.60,12.34,13.03,...,18.87,21.11,22.71,22.33,34.30,36.43,36.87,37.14,37.02,36.80
3,0010,18.25,19.57,21.23,20.97,19.91,17.98,17.70,22.17,25.36,...,71.55,93.33,122.22,185.36,212.88,230.02,251.68,266.48,303.04,320.15
4,0020,6.90,7.79,9.61,10.90,12.14,14.33,14.63,16.36,17.84,...,41.78,41.08,43.99,46.81,60.40,70.20,76.50,85.69,86.99,98.47
5,0050,9.95,14.40,19.83,22.06,27.24,30.17,36.26,39.20,43.10,...,279.51,265.08,244.40,210.57,159.80,174.38,173.05,172.42,174.36,194.50
6,0100,43.02,53.69,59.51,74.06,78.29,97.77,116.90,139.04,150.18,...,1354.44,1345.75,1370.28,1383.64,1402.43,1411.88,1442.10,1476.46,1502.65,1577.83
7,0500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2254.00,2631.08,3083.04,3644.79,4453.11,5128.07,5359.50,5702.48,6563.91,7853.75
8,1000,0.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,936.76,1412.19,1917.84,2382.52,3027.13,3468.81,4299.00,5081.37,5612.45,6325.68


In [64]:
dfWide['denom'] = dfWide.index

In [69]:
dfWide.index = ["notes","notes","notes","notes","notes","notes","notes","notes","notes"]

In [70]:
dfWide

year,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,...,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,denom
notes,2.72,2.56,2.14,2.07,2.12,2.05,1.98,2.04,2.38,3.01,...,3.00,3.00,2.99,2.99,2.99,2.99,2.99,2.99,3.09,0001
notes,1.48,1.74,2.15,2.99,3.92,4.31,4.50,5.80,6.50,6.83,...,6.36,6.65,6.98,8.51,8.51,8.51,8.51,8.54,8.53,0002
notes,5.72,5.76,5.88,6.34,6.92,8.07,5.60,12.34,13.03,12.87,...,21.11,22.71,22.33,34.30,36.43,36.87,37.14,37.02,36.80,0005
notes,18.25,19.57,21.23,20.97,19.91,17.98,17.70,22.17,25.36,25.10,...,93.33,122.22,185.36,212.88,230.02,251.68,266.48,303.04,320.15,0010
notes,6.90,7.79,9.61,10.90,12.14,14.33,14.63,16.36,17.84,18.99,...,41.08,43.99,46.81,60.40,70.20,76.50,85.69,86.99,98.47,0020
notes,9.95,14.40,19.83,22.06,27.24,30.17,36.26,39.20,43.10,52.47,...,265.08,244.40,210.57,159.80,174.38,173.05,172.42,174.36,194.50,0050
notes,43.02,53.69,59.51,74.06,78.29,97.77,116.90,139.04,150.18,171.16,...,1345.75,1370.28,1383.64,1402.43,1411.88,1442.10,1476.46,1502.65,1577.83,0100
notes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2631.08,3083.04,3644.79,4453.11,5128.07,5359.50,5702.48,6563.91,7853.75,0500
notes,0.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1412.19,1917.84,2382.52,3027.13,3468.81,4299.00,5081.37,5612.45,6325.68,1000


In [71]:
dfWide.to_csv("../data/notesWide.csv", index_label="index")